In [ ]:
# 패키지 및 데이터 로드
import pandas as pd
import numpy as np

Bike = pd.read_csv("/Users/yeondu/Downloads/final_final.csv", index_col = 0)
Bike.info()

In [ ]:
#데이터 전처리
Bike = Bike.drop('번호',axis=1)
Bike = Bike.drop('대여소명',axis=1)
Bike = Bike.drop('탄소량',axis=1)
Bike = Bike.drop('위도',axis=1)
Bike = Bike.drop('경도',axis=1)
Bike["대여시간"] = Bike["대여시간"].astype(str)
Bike["대여소번호"] = Bike["대여소번호"].astype(str)
Bike["운동량"][Bike["운동량"]== r"\N"] = -1
Bike["운동량"] = pd.to_numeric(Bike["운동량"])
Bike.info()

In [ ]:
#필요한 패키지 로드
import seaborn as sns 
import math
import matplotlib.pyplot as plt
import scipy

plt.style.use("seaborn")
sns.set(font_scale = 1)
sns.set_style("whitegrid")

import plotly.express as px
from scipy.stats import mode
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()

from plotly.subplots import make_subplots

plt.rcParams['font.family']='S-Core Dream'
plt.rcParams['axes.unicode_minus'] = False




In [ ]:
# 한글깨짐 해결
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont

        font_family = "S-Core Dream"
    return font_family
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family라는 변수에 할당합니다.
plt.style.use("ggplot")

font_family = get_font_family()
# 폰트설정
plt.rc("font", family=font_family)
# 마이너스 폰트 설정
plt.rc("axes", unicode_minus=False)
# 그래프에 retina display 적용
# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats("retina")

%config InlineBackend.figure_format = 'retina'



In [ ]:
#운동량 분포 시각화
import numpy as np
from scipy.stats import norm
#from matplotlib.mlab import norm.pdf ???doesn't work


mu = np.mean(Bike['운동량'])
#mu
sigma = np.std(Bike['운동량'])
#sigma
n_bins = 50
s = Bike['운동량']

fig, axes = plt.subplots(nrows=2, ncols=1, sharex = True)

#histogram
n, bins, patches = axes[1].hist(s, n_bins, density=True, alpha=0.1, edgecolor='black')
pdf = 1/(sigma*np.sqrt(2*np.pi))*np.exp(-(bins-mu)**2/(2*sigma**2))

median, q1, q3 = np.percentile(s, 50), np.percentile(s, 25), np.percentile(s, 75)
#print(q1, median, q3)

#probability density function
axes[1].plot(bins, pdf, color='orange', alpha=.6)

bins_1 = bins[(bins >= q1-1.5*(q3-q1)) & (bins <= q1)] #from Q1-1.5*IQR
bins_2 = bins[(bins <= q3+1.5*(q3-q1)) & (bins >= q3)]
pdf_1 = pdf[:int(len(pdf)/2)]
pdf_2 = pdf[int(len(pdf)/2):]
pdf_1 = pdf_1[(pdf_1 >= norm(mu,sigma).pdf(q1-1.5*(q3-q1))) & (pdf_1 <= norm(mu,sigma).pdf(q1))]
pdf_2 = pdf_2[(pdf_2 >= norm(mu,sigma).pdf(q3+1.5*(q3-q1))) & (pdf_2 <= norm(mu,sigma).pdf(q3))]

#fill from Q1-1.5*IQR to Q1 and Q3 to Q3+1.5*IQR
axes[1].fill_between(bins_1, pdf_1, 0, alpha=.6, color='orange')
axes[1].fill_between(bins_2, pdf_2, 0, alpha=.6, color='orange')

print(norm(mu, sigma).cdf(median))
print(norm(mu, sigma).pdf(median))


axes[1].annotate("{:.1f}%".format(100*norm(mu, sigma).cdf(q1)), xy=((q1-1.5*(q3-q1)+q1)/2, 0), ha='center')
axes[1].annotate("{:.1f}%".format(100*(norm(mu, sigma).cdf(q3)-norm(mu, sigma).cdf(q1))), xy=(median, 0), ha='center')
axes[1].annotate("{:.1f}%".format(100*(norm(mu, sigma).cdf(q3+1.5*(q3-q1)-q3)-norm(mu, sigma).cdf(q3))), xy=((q3+1.5*(q3-q1)+q3)/2, 0), ha='center')
axes[1].annotate('q1', xy=(q1, norm(mu, sigma).pdf(q1)), ha='center')
axes[1].annotate('q3', xy=(q3, norm(mu, sigma).pdf(q3)), ha='center')

axes[1].set_ylabel('probability')

#top boxplot
axes[0].boxplot(s, 0, 'gD', vert=False)
axes[0].axvline(median, color='orange', alpha=.6, linewidth=.5)
axes[0].axis('off')

plt.subplots_adjust(hspace=0)
plt.show()